In [1]:
import cirq
from collections import Counter

In [2]:
n = num_coins = 16
num_quibits = num_coins + 1
m = counterfeit_index = 7

assert num_coins > 2
assert 0 < counterfeit_index < num_coins

In [3]:
qbs = cirq.LineQubit.range(num_quibits)

circuit = cirq.Circuit(cirq.H.on_each(*qbs[:-1]))
circuit.append([cirq.CNOT(qbs[i], qbs[n]) for i in range(n)])
circuit.append([cirq.X(qbs[n]), cirq.measure(qbs[n], key="is_even")])

In [4]:
sub_circuit = cirq.Circuit()
sub_circuit.append(cirq.H(qbs[n]))
sub_circuit.append(cirq.CNOT(qbs[m], qbs[n]))
sub_circuit.append(cirq.H.on_each(*qbs[:-1]))
# execute subcircuit ONLY if `is_even` = 1
circuit.append(cirq.CircuitOperation(sub_circuit.freeze()).with_classical_controls("is_even"))
circuit.append([cirq.measure(qbs[i], key=f"qubit({i:02})") for i in range(n)])

In [5]:
simulator = cirq.Simulator()
results = simulator.run(circuit)
# counter for number of iterations algorithm is run
num_iterations = 1
while True:
    # repeat the algorithm until `is_even` is 1
    # ie, until we compute the desired superpostion over Q_e
    if results.measurements["is_even"][0] == 1:
        print("iteration:", num_iterations)
        print(results)
        break
    results = simulator.run(circuit)
    num_iterations += 1

iteration: 1
is_even=1
qubit(00)=1
qubit(01)=1
qubit(02)=1
qubit(03)=1
qubit(04)=1
qubit(05)=1
qubit(06)=1
qubit(07)=0
qubit(08)=1
qubit(09)=1
qubit(10)=1
qubit(11)=1
qubit(12)=1
qubit(13)=1
qubit(14)=1
qubit(15)=1


In [6]:
data = [results.data[f"qubit({i:02})"][0] for i in range(n)]
counts = Counter(data)

# find the least common measured value
least_common = min(counts, key=counts.get)
least_common_index = data.index(least_common)

assert counterfeit_index == least_common_index
print("index of counterfeit coin:", least_common_index)

index of counterfeit coin: 7


In [7]:
circuit

[ 0: ────H─────────── ]
                                                                                        [                     ]
                                                                                        [ 1: ────H─────────── ]
                                                                                        [                     ]
                                                                                        [ 2: ────H─────────── ]
                                                                                        [                     ]
                                                                                        [ 3: ────H─────────── ]
                                                                                        [                     ]
                                                                                        [ 4: ────H─────────── ]
                                                                                        [                     ]
                                                                                        [ 5: ────H─────────── ]
                                                                                        [                     ]
                                                                                        [ 6: ────H─────────── ]
                                                                                        [                     ]
                                                                                        [ 7: ────────@───H─── ]
                                                                                        [            │        ]
0: ─────────H───@───────────────────────────────────────────────────────────────────────[ 8: ────H───┼─────── ]────────────────────────────────────M('qubit(00)')───
                │                                                                       [            │        ]
                │                                                                       [ 9: ────H───┼─────── ]
                │                                                                       [            │        ]
                │                                                                       [ 10: ───H───┼─────── ]
                │                                                                       [            │        ]
                │                                                                       [ 11: ───H───┼─────── ]
                │                                                                       [            │        ]
                │                                                                       [ 12: ───H───┼─────── ]
                │                                                                       [            │        ]
                │                                                                       [ 13: ───H───┼─────── ]
                │                                                                       [            │        ]
                │                                                                       [ 14: ───H───┼─────── ]
                │                                                                       [            │        ]
                │                                                                       [ 15: ───H───┼─────── ]
                │                                                                       [            │        ]
                │                                                                       [ 16: ───H───X─────── ].with_classical_controls(is_even)
                │                                                                       ║
1: ─────────H───┼───@───────────────────────────────────────────────────────────────────#2─────────────────────────────────────────────────────────M('qubit(01)')───
                │   │                              